## Spatial Pooler（空間プーリング）

In [ ]:
from nupic.algorithms.spatial_pooler import SpatialPooler

In [ ]:
from nupic.encoders.category import CategoryEncoder

categories = ("cat", "dog", "monkey", "slow loris")
encoder = CategoryEncoder(w=3, categoryList=categories, forced=True)
cat = encoder.encode("cat")
dog = encoder.encode("dog")
monkey = encoder.encode("monkey")
loris = encoder.encode("slow loris")
print "cat =       ", cat
print "dog =       ", dog
print "monkey =    ", monkey
print "slow loris =", loris

In [ ]:
print len(cat)
print cat

**`SpatialPooler(...)`**

空間プーラーを実装したクラスである。領域のカラムと入力ビットの関係を扱う。

この関数の主要なパブリック・インタフェースは "compute" メソッドであり、入力ベクトルを受け取り、activeColumns カラムのリストを返す。

```
sp = SpatialPooler(...)
for line in file:
  inputVector = numpy.array(line)
  sp.compute(inputVector)
  ...
```

**パラメータ**
- `inputDimensions` - (int) 入力ベクトルの次元を表すシーケンス
- `columnDimensions` - (int) リージョン内のカラムの寸法を表すシーケンス
- `potentialRadius` - (int) 各カラムが潜在的に接続できる入力の範囲を決定します
- `numActiveColumnsPerInhArea` - (float) アクティブなカラムの密度を制御する別の方法
- `globalInhibition` - (bool) true の場合、抑制フェーズの間、勝利したカラムは、リージョン全体の中で最もアクティブなカラムとして選択されます
- `synPermActiveInc` - (float) アクティブなシナプスが各ラウンドで増加する量
- `potentialPct` - (float) カラムのポテンシャル半径内で、カラムが接続できる入力のパーセンテージ

**`getConnectedSynapses(columnIndex, connectedSynapses)`**

**パラメータ**
- connectedSynapses -  (list)は上書きされます。

**戻り値**
- (iter) 指定されたカラムに接続されているシナプスの数を返す。

In [ ]:
sp = SpatialPooler(inputDimensions=(15,),
                   columnDimensions=(4,),
                   potentialRadius=15,
                   numActiveColumnsPerInhArea=1,
                   globalInhibition=True,
                   synPermActiveInc=0.03,
                   potentialPct=1.0)

import numpy
for column in xrange(4):
    connected = numpy.zeros((15,), dtype="int")
    sp.getConnectedSynapses(column, connected)
    print connected

**`compute(inputVector, learn, activeArray)`**

これは SpatialPooler クラスの主要なパブリック・メソッドです。
この関数は入力ベクトルを受け取り、アクティブなカラムのインデックスを出力します。
`learn` が True に設定されている場合、このメソッドはカラムの永続性も更新します。

**パラメーター**

- `inputVector` - 空間プーラーへの入力を構成する0と1のnumpy配列．配列は1次元配列として扱われるので，配列の次元はクラスのコンストラクタで指定された次元と正確に一致する必要はありません．

- `learn` - 学習を行うかどうかを示すブール値．学習はシナプスの永続値を更新し、モデルの「状態」を修正することを意味する

- `activeArray` - カラム数と同じサイズの配列。関数が返す前に、この配列にはアクティブなカラムのインデックスに 1 が、それ以外の場所には 0 が格納されます。

In [ ]:
output = numpy.zeros((4,), dtype="int")
sp.compute(cat, learn=True, activeArray=output)
print output

`xrange(20)`20回学習

In [ ]:
for _ in xrange(20):
    sp.compute(cat, learn=True, activeArray=output)

for column in xrange(4):
    connected = numpy.zeros((15,), dtype="int")
    sp.getConnectedSynapses(column, connected)
    print connected

`xrange(200)`200回学習

In [ ]:
for _ in xrange(200):
    sp.compute(cat, learn=True, activeArray=output)
    sp.compute(dog, learn=True, activeArray=output)
    sp.compute(monkey, learn=True, activeArray=output)
    sp.compute(loris, learn=True, activeArray=output)

In [ ]:
for column in xrange(4):
    connected = numpy.zeros((15,), dtype="int")
    sp.getConnectedSynapses(column, connected)
    print connected

In [ ]:
noisyCat = numpy.zeros((15,), dtype="uint32")
noisyCat[3] = 1
noisyCat[4] = 1
# This is part of dog!
noisyCat[6] = 1
print noisyCat

In [ ]:
sp.compute(noisyCat, learn=False, activeArray=output)
print output  # matches cat!